In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/interbank20/sample_submission.csv
/kaggle/input/interbank20/y_train.csv
/kaggle/input/interbank20/sunat_test.csv
/kaggle/input/interbank20/rcc_test.csv
/kaggle/input/interbank20/se_test.csv
/kaggle/input/interbank20/sunat_train.csv
/kaggle/input/interbank20/se_train.csv
/kaggle/input/interbank20/censo_test.csv
/kaggle/input/interbank20/productos.csv
/kaggle/input/interbank20/censo_train.csv
/kaggle/input/interbank20/rcc_train.csv
/kaggle/input/script-final-1/__results__.html
/kaggle/input/script-final-1/__notebook__.ipynb
/kaggle/input/script-final-1/__output__.json
/kaggle/input/script-final-1/train_complete.csv
/kaggle/input/script-final-1/test_complete.csv
/kaggle/input/script-final-1/custom.css


In [2]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import re
import os
import warnings

#basic tools 
import os
import warnings

#tuning hyperparameters
from bayes_opt import BayesianOptimization
from skopt  import BayesSearchCV 

#graph, plots
import matplotlib.pyplot as plt
import seaborn as sns

#building models
import lightgbm as lgb
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import time
import sys

#metrics 
from sklearn.metrics import roc_curve
import shap
warnings.simplefilter(action='ignore', category=FutureWarning)

Creo una funcion para leer los datasets usando menos memoria:

In [3]:
#Funcion:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


Leo los datasets creados anteriormente en el script_1 más los datasets RCC con la función reduce_mem_usage:

In [4]:
train = reduce_mem_usage(pd.read_csv("/kaggle/input/script-final-1/train_complete.csv"))
test = reduce_mem_usage(pd.read_csv("/kaggle/input/script-final-1/test_complete.csv"))

rcc_train = reduce_mem_usage(pd.read_csv("/kaggle/input/interbank20/rcc_train.csv"))
rcc_test = reduce_mem_usage(pd.read_csv("/kaggle/input/interbank20/rcc_test.csv"))

y_train = pd.read_csv("/kaggle/input/interbank20/y_train.csv", index_col="key_value").target



Mem. usage decreased to 836.24 Mb (75.0% reduction)
Mem. usage decreased to 889.74 Mb (75.0% reduction)
Mem. usage decreased to 521.71 Mb (75.0% reduction)
Mem. usage decreased to 622.45 Mb (73.6% reduction)


Creo variables nuevas:

In [5]:
train["gastos"]=train["ctd_hijos"]+train["ctd_veh"]
test["gastos"]=test["ctd_hijos"]+test["ctd_veh"]

train["sum_saldo"]=train.loc[:, train.columns.str.startswith('condicion_sum')].sum(axis=1)
test["sum_saldo"]=test.loc[:, test.columns.str.startswith('condicion_sum')].sum(axis=1)

agrupado=rcc_train.groupby(["key_value"])
train["sum_cond"]=agrupado.condicion.sum()

agrupado=rcc_test.groupby(["key_value"])
test["sum_cond"]=agrupado.condicion.sum()

train["saldo/cond"]=train["sum_saldo"]/train["sum_cond"]
test["saldo/cond"]=test["sum_saldo"]/test["sum_cond"]

agrupado=rcc_train.groupby(["key_value"])
train["sum_riesgo_1"]=agrupado.RIESGO_DIRECTO.sum()
agrupado=rcc_train.groupby(["key_value"])
train["sum_riesgo_2"]=agrupado.COD_CLASIFICACION_DEUDOR.sum()
agrupado=rcc_test.groupby(["key_value"])
test["sum_riesgo_1"]=agrupado.RIESGO_DIRECTO.sum()
agrupado=rcc_test.groupby(["key_value"])
test["sum_riesgo_2"]=agrupado.COD_CLASIFICACION_DEUDOR.sum()

train["saldo/riesgo_1"]=train["sum_saldo"]/train["sum_riesgo_1"]
train["saldo/riesgo_2"]=train["sum_saldo"]/train["sum_riesgo_2"]
test["saldo/riesgo_1"]=test["sum_saldo"]/test["sum_riesgo_1"]
test["saldo/riesgo_2"]=test["sum_saldo"]/test["sum_riesgo_2"]


Limpio memoria:

In [6]:
import gc
del rcc_train, rcc_test
gc.collect()

32

Me aseguro que train y test tengan las mismas variables, y renombro.

In [7]:
keep_cols = list(set(train.columns).intersection(set(test.columns)))
train = train[keep_cols]
test = test[keep_cols]
len(set(train.columns) - set(test.columns)) , len(set(test.columns) - set(train.columns))

(0, 0)

In [8]:
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))

-----------------------------------------------------

MODELO

Corro una optimización bayesiana para encontrar los hyperparámetros óptimos de LGBM:


In [9]:
def bayes_parameter_opt_lgb(train, y_train, init_round=15, opt_round=25, n_folds=5, random_seed=6,n_estimators=10000, output_process=False):
    # prep data:
    train_data = lgb.Dataset(data=train, label=y_train, free_raw_data=False)
    # parameters
    def lgb_eval(learning_rate,num_leaves, feature_fraction, bagging_fraction, max_depth, max_bin, min_data_in_leaf,min_sum_hessian_in_leaf,min_child_samples,subsample):
        params = {'application':'binary', 'metric':'auc'}
        params['learning_rate'] = max(min(learning_rate, 1), 0)
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['max_bin'] = int(round(max_depth))
        params['min_child_samples'] = int(round(min_child_samples))
        params['min_data_in_leaf'] = int(round(min_data_in_leaf))
        params['min_sum_hessian_in_leaf'] = min_sum_hessian_in_leaf
        params['subsample'] = max(min(subsample, 1), 0)
        
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return max(cv_result['auc-mean'])
     
    lgbBO = BayesianOptimization(lgb_eval, {'learning_rate': (0.01, 1.0),
                                            'num_leaves': (50, 700),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.1, 1),
                                            'max_depth': (5, 30),
                                            'max_bin':(20,90),
                                            'min_data_in_leaf': (100,1000),
                                            'min_sum_hessian_in_leaf':(0,100),
                                            'min_child_samples':(100,1500),
                                            'subsample':(0.01, 1.0)}, random_state=200)
    
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    model_auc=[]
    for model in range(len( lgbBO.res)):
        model_auc.append(lgbBO.res[model]['target'])
    
    # mejores parametros:
    return lgbBO.res[pd.Series(model_auc).idxmax()]['target'],lgbBO.res[pd.Series(model_auc).idxmax()]['params']

opt_params = bayes_parameter_opt_lgb(train, y_train, init_round=5, opt_round=10, n_folds=5, random_seed=6,n_estimators=10000)


|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | max_depth | min_ch... | min_da... | min_su... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] bagging_fraction is set=0.9528690311665032, subsample=0.9819846820667453 will be ignored. Current value: bagging_fraction=0.9528690311665032
[LightGBM] [Warning] min_data_in_leaf is set=422, min_child_samples=104 will be ignored. Current value: min_data_in_leaf=422
[LightGBM] [Warning] bagging_fraction is set=0.9528690311665032, subsample=0.9819846820667453 will be ignored. Current value: bagging_fraction=0.9528690311665032
[LightGBM] [Warning] min_data_in_leaf is set=422, min_child_samples=104 will be ignored. Current value: min_data_in_leaf=422
[LightGBM] [Warning] bagging_fraction is set=0.9528690311665032, subsample=0.9819846820667453 will be ignored. Current value: bagging_

Hyperparametros optimos:

In [10]:
opt_params[1]["num_leaves"] = int(round(opt_params[1]["num_leaves"]))
opt_params[1]['max_depth'] = int(round(opt_params[1]['max_depth']))
opt_params[1]['min_data_in_leaf'] = int(round(opt_params[1]['min_data_in_leaf']))
opt_params[1]['max_bin'] = int(round(opt_params[1]['max_bin']))
opt_params[1]['objective']='binary'
opt_params[1]['metric']='auc'
opt_params[1]['is_unbalance']=True
opt_params[1]['boost_from_average']=False
opt_params=opt_params[1]
opt_params


{'bagging_fraction': 0.21792453148378854,
 'feature_fraction': 0.47611446763369825,
 'learning_rate': 0.14674002393101493,
 'max_bin': 56,
 'max_depth': 16,
 'min_child_samples': 916.2326894189557,
 'min_data_in_leaf': 471,
 'min_sum_hessian_in_leaf': 61.2284698145987,
 'num_leaves': 88,
 'subsample': 0.5418669064457182,
 'objective': 'binary',
 'metric': 'auc',
 'is_unbalance': True,
 'boost_from_average': False}

Entreno el modelo LGBM, usando cross valdation y los hyperparametros encontrados. Genero la predicción.

In [11]:
folds = [train.index[t] for t, v in KFold(5).split(train)]


from sklearn.model_selection import ParameterGrid
params = opt_params
best_score = 0
best_probs = []
test_probs = []
train_probs = []
for i, idx in enumerate(folds):
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]

    learner = LGBMClassifier(n_estimators=1000, **params)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=False)
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
score = roc_auc_score(y_train, train_probs.loc[y_train.index])
print(f"roc auc estimado para: {score}")
if score > best_score:
    best_score = score
    best_probs = test_probs


best_probs.name = "target"
best_probs.index.set_names("key_value", level=None, inplace=True)
best_probs.to_csv("scriptbar_FINAL.csv")


[LightGBM] [Warning] bagging_fraction is set=0.21792453148378854, subsample=0.5418669064457182 will be ignored. Current value: bagging_fraction=0.21792453148378854
[LightGBM] [Warning] feature_fraction is set=0.47611446763369825, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.47611446763369825
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=61.2284698145987, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=61.2284698145987
[LightGBM] [Warning] min_data_in_leaf is set=471, min_child_samples=916.2326894189557 will be ignored. Current value: min_data_in_leaf=471
[LightGBM] [Warning] bagging_fraction is set=0.21792453148378854, subsample=0.5418669064457182 will be ignored. Current value: bagging_fraction=0.21792453148378854
[LightGBM] [Warning] feature_fraction is set=0.47611446763369825, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.47611446763369825
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=61.